# Fitting Encoding Models with NEMS Library

In this tutotial we show how to easily train models built using the [Neural Encoding Model System (NEMS)](https://github.com/LBHB/NEMS) library. Here, we show how to train cross-validated linear-nonlinear STRF models.

In [38]:
import numpy as np
from scipy.signal import resample
from tqdm import tqdm

import naplib as nl
from naplib.model_selection import KFold

In [29]:
from nems import Model
from nems.layers import FIR, DoubleExponential
from nems.models import LN_STRF
from nems.metrics import correlation

In [30]:
data = nl.io.load_speech_task_data()
print(f'This data has {len(data)} trials')

This data has 10 trials


In [31]:
# look at the shape of the spectrograms we have
print(data['aud'][0].shape) # (time * frequency)

(6197, 128)


In [32]:
# resample spectrograms to be fewer frequency bins
data['aud'] = [resample(x, 24, axis=1) for x in data['aud']]
print(data['aud'][0].shape) # (time * frequency)

(6197, 24)


In [33]:
# look at the shape of the responses we have
print(data['resp'][0].shape) # (time * channels)

# for simplicity and time-saving, let's only use the first 3 channels
data['resp'] = [x[:,:3] for x in data['resp']]
print(data['resp'][0].shape) # (time * channels)


(6197, 10)
(6197, 3)


In [34]:
# normalize responses
data['resp'] = nl.preprocessing.normalize(data=data)

## Define and fit a simple Linear-Nonlinear (LN) model with NEMS

This example model consists of a STRF component followed by a double-exponential nonlinearity.

In [35]:
# fitting options (quick fitting for the sake of this tutorial notebook)
options = {'options': {'maxiter': 2, 'ftol': 1e-1}}

### Fit the model using cross-validation

In [41]:
kfold = nl.model_selection.KFold(5) # 5-fold cross validation
predictions = []
scores = []

for i, (data_train, data_test) in tqdm(enumerate(kfold.split(data))):
    spec_train = np.concatenate(data_train['aud'], axis=0) # shape (time * frequency)
    spec_test = np.concatenate(data_test['aud'], axis=0) # shape (time * frequency)
    resp_train = np.concatenate(data_train['resp'], axis=0) # shape (time * channels)
    resp_test = np.concatenate(data_test['resp'], axis=0) # shape (time * channels)
    
    # define LN model
    model = Model()
    model.add_layers(
        FIR(shape=(25,24)),    # Full-rank STRF, 25 temporal bins x 23 spectral channels
        DoubleExponential(shape=(3,)) # Double-exponential nonlinearity
    )
    
    model.fit(input=spec_train, target=resp_train, fitter_options=options)
    
    predictions.append(model.predict(input=spec_test))

    scores.append(correlation(predictions[-1], resp_test))
    

0it [00:00, ?it/s]

Epoch 0
        Iteration 0, error is: 1.06154751...
Fit successful: True
Status: 0
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH


/Users/gavin/anaconda3/envs/naplib-nems/lib/python3.9/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  .. [2] E.R. Kanasewich, "Time Sequence Analysis in Geophysics",
/Users/gavin/anaconda3/envs/naplib-nems/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  The University of Alberta Press, 1975, pp. 109-110.
1it [05:32, 332.20s/it]

Epoch 0
        Iteration 0, error is: 1.05721753...
Fit successful: True
Status: 0
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH


2it [10:39, 317.54s/it]

...

3it [15:31, 306.01s/it]

Epoch 0
        Iteration 0, error is: 1.06451277...
Fit successful: True
Status: 0
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH


4it [20:11, 295.56s/it]

Epoch 0
        Iteration 0, error is: 1.02326504...
Fit successful: True
Status: 0
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH


5it [26:16, 315.30s/it]


In [43]:
predictions_full = np.concatenate(predictions, axis=0)
resp_full = np.concatenate(data['resp'], axis=0)
predictions_full.shape, resp_full.shape

((64441, 3), (64441, 3))